In [1]:
from random import Random
import numpy as np
import pandas as pd
from tva.situation import Situation
from tva.strategies import Strategies
import copy
from tva.schemes import Schemes, VotingScheme
from tva.enums import VotingScheme, Happiness
from tva.voter import Voter

In [2]:
# Create the voting situation
situation = Situation(num_voters=4, num_candidates=4, seed=42)
display(situation)

# Apply the voting scheme to the situation
schemes = Schemes()
voting_scheme = VotingScheme.BORDA
schemes.print_results(situation, verbose=True)



Voter 0: ['A', 'C', 'D', 'B']
Voter 1: ['D', 'A', 'B', 'C']
Voter 2: ['C', 'A', 'D', 'B']
Voter 3: ['A', 'B', 'C', 'D']

Anti plurality: A {'A': 4, 'C': 3, 'D': 3, 'B': 2} 
Two voting: A {'A': 4, 'C': 2, 'D': 1, 'B': 1} 
Borda: A {'A': 10, 'C': 6, 'D': 5, 'B': 3}


In [8]:
situation = Situation(num_voters=4, num_candidates=4)
situation.voters[0].preferences = ["A","B","C","D"]
situation.voters[1].preferences = ["C","A","B","D"]
situation.voters[2].preferences = ["D","B","C","A"]
situation.voters[3].preferences = ["D","B","A","C"]
display(situation)

current_winner, scores = schemes.apply_voting_scheme(VotingScheme.BORDA, situation.voters, return_scores=True)
print("Borda:", current_winner, scores)

# situation.voters[0].preferences = ["B","C","A","D"]
# current_winner, scores = schemes.apply_voting_scheme(VotingScheme.BORDA, situation.voters, return_scores=True)
# print("Borda:", current_winner, scores)

strategies = Strategies()
a = strategies.bury(situation, 0, VotingScheme.BORDA, Happiness.LINEAR, verbose=True, exhaustive_search=True)
print(a)

Voter 0: ['A', 'B', 'C', 'D']
Voter 1: ['C', 'A', 'B', 'D']
Voter 2: ['D', 'B', 'C', 'A']
Voter 3: ['D', 'B', 'A', 'C']

Borda: B {'A': 6, 'B': 7, 'C': 5, 'D': 6}
['A', 'B', 'C', 'D']
Borda: B {'A': 6, 'B': 7, 'C': 5, 'D': 6}
Swapping B and C
['A', 'C', 'B', 'D']
Borda: A {'A': 6, 'C': 6, 'B': 6, 'D': 6}
Found a winning strategy!
Swapping B and D
['A', 'C', 'D', 'B']
Borda: D {'A': 6, 'C': 6, 'D': 7, 'B': 5}
Winner changed
Swapping D and B
['A', 'C', 'B', 'D']
Loop detected
Left recursion
Left recursion
None


In [ ]:
import copy

def get_indexes_to_iterate(preferences:list[str], scores:dict[str,int]):
    # Given a list of preferences and the scores of those preferences, return a list of indexes of candidates to the left of the current winner
    # The current winner is the candidate with the highest score
    # Return the indexes of preferences sorted by score in descending order
    # If the current winner is at index 0, return an empty list
    current_winner = max(scores, key=scores.get) # type: ignore
    current_winner_index = preferences.index(current_winner)

    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    print("Sorted scores:", sorted_scores)
    candidates = [x[0] for x in sorted_scores[1:]]
    print("Candidates:", candidates)

    indexes_to_try = []
    for i in candidates:
        candidate_index = preferences.index(i)
        if candidate_index < current_winner_index:
            indexes_to_try.append(candidate_index)
    return indexes_to_try

def swap(situation:Situation, voter_id:int, candidate1_index:int, candidate2_index:int, verbose=False):
    if verbose:
        print(f"Swapping {situation.voters[voter_id].preferences[candidate1_index]} and {situation.voters[voter_id].preferences[candidate2_index]}")
    situation.voters[voter_id].preferences[candidate1_index], situation.voters[voter_id].preferences[candidate2_index] = situation.voters[voter_id].preferences[candidate2_index], situation.voters[voter_id].preferences[candidate1_index]

def compromise(situation:Situation, voter_index:int, voting_scheme:VotingScheme, happiness_func:Happiness, exhaustive_search=False, verbose=False) -> None | list[str]:
    # Copy the situation
    new_situation = copy.deepcopy(situation)
    original_voter = new_situation.voters[voter_index]
    original_preferences = new_situation.voters[voter_index].preferences
    # If the original winner is the first preference of the voter, return False
    original_winner, scores = schemes.apply_voting_scheme(voting_scheme, new_situation.voters, return_scores=True)
    original_winner_index = original_preferences.index(original_winner) # type: ignore
    original_winner_happiness = original_voter.calculate_happiness(original_winner, happiness_func)

    if verbose:
        print(original_preferences)
        print("Borda:", original_winner, scores)

    if original_winner_index == 0:
        return None
    
    all_winning_preferences = []
    indexes_to_iterate = get_indexes_to_iterate(original_preferences, scores) # type: ignore
    # Loop over all candidates to the left of the original winner sorted by score
    for i in indexes_to_iterate:
        # Move that candidate to the first position
        swap(new_situation, voter_index, i, 0, verbose=verbose)
        # Check if the winner changed and if the voter is happier
        new_winner, scores = schemes.apply_voting_scheme(voting_scheme, new_situation.voters, return_scores=True)
        new_winner_happiness = original_voter.calculate_happiness(new_winner, happiness_func)

        if verbose:
            print(new_situation.voters[voter_index].preferences)
            print("Borda:", new_winner, scores)

        if new_winner != original_winner and new_winner_happiness > original_winner_happiness:
            if exhaustive_search:
                all_winning_preferences.append(new_situation.voters[voter_index].preferences)
                print("Found a winning preference")
            else:
                print("Found a winning preference")
                return new_situation.voters[voter_index].preferences

    if all_winning_preferences == []:
        return None
    return all_winning_preferences

compromise(situation, 0, VotingScheme.BORDA, Happiness.LINEAR, verbose=True, exhaustive_search=True)

['A', 'D', 'C', 'B']
Borda: D {'A': 4, 'D': 10, 'C': 5, 'B': 5}
Sorted scores: [('D', 10), ('C', 5), ('B', 5), ('A', 4)]
Candidates: ['C', 'B', 'A']
Swapping A and A
['A', 'D', 'C', 'B']
Borda: D {'A': 4, 'D': 10, 'C': 5, 'B': 5}


Create 1000 situations and check for what fraction of them a strategy is good

We do experiments for each voting scheme separately.<br>
TODO: We need to save, what type of strategy has won this time?

In [1]:
strategies = Strategies()
voting_scheme = VotingScheme.BORDA
situation = Situation(num_voters=5, num_candidates=4)
situation

NameError: name 'Strategies' is not defined

In [33]:
strategies = Strategies()
voting_scheme = VotingScheme.BORDA
nr_repetitions = 1000
num_voters = 4

strategy_counter = 0
for i in range(nr_repetitions):
    situation = Situation(num_voters=num_voters, num_candidates=4)
    # Check if at least one voter has a good strategy
    for voter_index in range(num_voters):
        if strategies.is_any_strategy_good(situation, voter_index, voting_scheme):
            strategy_counter += 1
            break
        
print(strategy_counter/nr_repetitions)

0.877


Plurality cannot be affected by strategic voting because that would require a voter to vote for a candidate they prefer less than their favorite candidate.